In [1]:
import yfinance as yf
import pandas as pd
import warnings
import datetime

# Ignore warnings
warnings.filterwarnings('ignore')

# Load stock list from Excel file
excel_file_path = 'Short_List_Twenty1 .xlsx'
sheet_name = 'Short_List_Twenty'

# Read the Excel file into a DataFrame
stock_list_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Extract the 'Ticker' column and convert it to a list
indian_stock_list = stock_list_df['Ticker'].tolist()

# Function to check stock details and identify profitable stocks
def check_stock(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="max", interval="1d")
    keys_to_extract = [
        'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'maxAge', 'priceHint',
        'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 
        'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'payoutRatio', 'beta', 'trailingPE', 
        'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 
        'bid', 'ask', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 
        'fiftyDayAverage', 'twoHundredDayAverage', 'enterpriseValue', 'profitMargins', 'floatShares', 
        'sharesOutstanding', 'heldPercentInsiders', 'heldPercentInstitutions', 'impliedSharesOutstanding', 
        'bookValue', 'priceToBook', 'lastFiscalYearEnd', 'nextFiscalYearEnd', 'mostRecentQuarter', 
        'earningsQuarterlyGrowth', 'netIncomeToCommon', 'trailingEps', 'forwardEps', 'lastSplitFactor', 
        'lastSplitDate', 'enterpriseToRevenue', 'enterpriseToEbitda', '52WeekChange', 'SandP52WeekChange', 
        'lastDividendValue', 'lastDividendDate', 'shortName', 'longName', 'firstTradeDateEpochUtc', 
        'gmtOffSetMilliseconds', 'currentPrice', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 
        'targetMedianPrice', 'recommendationMean', 'recommendationKey', 'numberOfAnalystOpinions', 'totalCash', 
        'totalCashPerShare', 'ebitda', 'totalDebt', 'totalRevenue', 'debtToEquity', 'revenuePerShare', 
        'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins'
    ]
    All_data = {}
    try:
        fifty_two_week_high = stock.info['fiftyTwoWeekHigh']
    except KeyError:
        fifty_two_week_high = 0

    if hist.empty or len(hist) < 2:
        return None

    current_price = hist['Close'][-1]
    all_time_high_till_yesterday = hist['High'][:-1].max()
    todays_high = hist['High'][-1]
    todays_open = hist['Open'][-1]

    breakout_level = fifty_two_week_high if all_time_high_till_yesterday > fifty_two_week_high and fifty_two_week_high != 0 else all_time_high_till_yesterday

    if 50 < current_price < 3000 and todays_high >= breakout_level + 1:
        volume = hist['Volume'][-1]

        if todays_open > breakout_level:
            entry_price = todays_open
            stop_loss = breakout_level - 1
        else:
            entry_price = breakout_level
            stop_loss = max(todays_open, breakout_level * 0.99) - 1

        exit_price = max(current_price, stop_loss)
        stop_loss_hit = "Yes" if current_price < stop_loss else "No"
        profit = exit_price - entry_price
        extracted_values = {key: stock.info.get(key) for key in keys_to_extract}
        All_data.update(extracted_values)

        data = {
            'Ticker': ticker,
            'All Time High': all_time_high_till_yesterday,
            '52 Week High': fifty_two_week_high,
            'Breakout Level': breakout_level,
            'Today\'s High': todays_high,
            'Trailing Stop Loss': todays_high - 5,
            'Today\'s Open': todays_open,
            'Current Market Price': current_price,
            'Stop Loss': stop_loss,
            'Stop Loss Points': entry_price - stop_loss,
            'Stop Loss Hit': stop_loss_hit,
            'Volume': volume,
            'Entry Price': entry_price,
            'Exit Price': exit_price,
            'Profit': profit
        }
        All_data.update(data)

    return All_data

# List to store filtered stocks
filtered_stocks = []

# Loop through the tickers and apply the filter
for ticker in indian_stock_list:
    result = check_stock(ticker)
    if result:
        filtered_stocks.append(result)

# Create a DataFrame from the results
filtered_stocks_df = pd.DataFrame(filtered_stocks)

# Sort the DataFrame by Volume in descending order
filtered_stocks_df = filtered_stocks_df.sort_values(by='Volume', ascending=False)

# Calculate the total profit and other metrics
total_profit = filtered_stocks_df['Profit'].sum()
total_trailing_sl = filtered_stocks_df['Trailing Stop Loss'].sum()
total_entry_capital = filtered_stocks_df['Entry Price'].sum()
total_return_percentage = (total_profit / total_entry_capital) * 100
leverage_capital = total_entry_capital * 0.2
total_return_with_leverage = (total_profit / leverage_capital) * 100

# Print the DataFrame and calculated metrics
if not filtered_stocks_df.empty:
    print(filtered_stocks_df)
    print(f"\nToday's Profit: {total_profit}")
    print(f"\nTrailing Stop Loss High Profit: {total_trailing_sl}")
    print(f"\nTotal Capital Used: {total_entry_capital}")
    print(f"\nTotal Return for the Day: {total_return_percentage:.2f}%")
    print(f"\nWith 80% Leverage, Capital Used: {leverage_capital}")
    print(f"\nTotal Return for the Day with Leverage: {total_return_with_leverage:.2f}%")

    timestamp = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M")
    filtered_stocks_df.to_csv(f'strategy_results_{timestamp}.csv', index=False)
else:
    print("No stocks meet the criteria.")

   auditRisk  boardRisk  compensationRisk  shareHolderRightsRisk  overallRisk  \
0          2          7                 7                      1            5   

   maxAge  priceHint  previousClose   open  dayLow  ...  Trailing Stop Loss  \
0   86400          2         232.43  235.1   235.1  ...          243.889999   

   Today's Open  Current Market Price   Stop Loss  Stop Loss Points  \
0    235.100006            246.649994  242.213298            3.4567   

   Stop Loss Hit    Volume  Entry Price  Exit Price    Profit  
0             No  41718501   245.669998  246.649994  0.979996  

[1 rows x 96 columns]

Today's Profit: 0.9799957275390625

Trailing Stop Loss High Profit: 243.88999938964844

Total Capital Used: 245.6699981689453

Total Return for the Day: 0.40%

With 80% Leverage, Capital Used: 49.13399963378907

Total Return for the Day with Leverage: 1.99%


In [11]:
stock = yf.Ticker("ASHOKLEY.NS")
hist = stock.info
keys_to_extract = [
    'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'maxAge', 'priceHint',
    'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 
    'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'payoutRatio', 'beta', 'trailingPE', 
    'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 
    'bid', 'ask', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 
    'fiftyDayAverage', 'twoHundredDayAverage', 'enterpriseValue', 'profitMargins', 'floatShares', 
    'sharesOutstanding', 'heldPercentInsiders', 'heldPercentInstitutions', 'impliedSharesOutstanding', 
    'bookValue', 'priceToBook', 'lastFiscalYearEnd', 'nextFiscalYearEnd', 'mostRecentQuarter', 
    'earningsQuarterlyGrowth', 'netIncomeToCommon', 'trailingEps', 'forwardEps', 'lastSplitFactor', 
    'lastSplitDate', 'enterpriseToRevenue', 'enterpriseToEbitda', '52WeekChange', 'SandP52WeekChange', 
    'lastDividendValue', 'lastDividendDate', 'shortName', 'longName', 'firstTradeDateEpochUtc', 
    'gmtOffSetMilliseconds', 'currentPrice', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 
    'targetMedianPrice', 'recommendationMean', 'recommendationKey', 'numberOfAnalystOpinions', 'totalCash', 
    'totalCashPerShare', 'ebitda', 'totalDebt', 'totalRevenue', 'debtToEquity', 'revenuePerShare', 
    'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins'
]


# Extract the key values
extracted_values = {key: hist.get(key) for key in keys_to_extract}

# Display the extracted values
for key, value in extracted_values.items():
    print(f"{key}: {value}")

auditRisk: 2
boardRisk: 7
compensationRisk: 7
shareHolderRightsRisk: 1
overallRisk: 5
maxAge: 86400
priceHint: 2
previousClose: 232.43
open: 235.1
dayLow: 235.1
dayHigh: 248.89
regularMarketPreviousClose: 232.43
regularMarketOpen: 235.1
regularMarketDayLow: 235.1
regularMarketDayHigh: 248.89
dividendRate: 7.55
payoutRatio: 0.59419996
beta: 0.904
trailingPE: 29.657454
forwardPE: 31.838709
volume: 40458038
regularMarketVolume: 40458038
averageVolume: 17564331
averageVolume10days: 12428060
averageDailyVolume10Day: 12428060
bid: 246.78
ask: 246.85
marketCap: 724625784832
fiftyTwoWeekLow: 157.55
fiftyTwoWeekHigh: 248.89
priceToSalesTrailing12Months: 1.5475583
fiftyDayAverage: 228.702
twoHundredDayAverage: 189.0915
enterpriseValue: 1050009731072
profitMargins: 0.0523
floatShares: 1739509642
sharesOutstanding: 2936430080
heldPercentInsiders: 0.52731
heldPercentInstitutions: 0.22681
impliedSharesOutstanding: 2936679936
bookValue: 30.666
priceToBook: 8.0463705
lastFiscalYearEnd: 1711843200
next